<a href="https://colab.research.google.com/github/hskim-py/Data-Analysis-with-Open-Source/blob/main/Ansys_motion_post_result_FFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Using Python.NET
import clr, sys
import matplotlib.pyplot as plt

sys.path.append(r'C:\Program Files\ANSYS Inc\v242\Motion\bin')
clr.AddReference(r'C:\Program Files\ANSYS Inc\v242\Motion\bin\VM.Post.API.OutputReader.dll')
clr.AddReference(r'C:\Program Files\ANSYS Inc\v242\Motion\bin\VM.Models.dll')


from VM.Post.API.OutputReader import *
from VM.Models.OutputReader import *
from VM.Models import *
from VM.Models.Post import *

#targets_paths = {"TIE_ROD_R":"Acceleration/X", "TIE_ROD_R":"Acceleration/Y", "TIE_ROD_R":"Acceleration/Z"}

from System.Collections.Generic import List


# Import result file

#outputReader = OutputReader(r"E:\00_Project\RBNA_NVH_MODEL\R&V_WOW_BOB_Correlation\JX_deflector_angle_fc_RG150_WOW3.dfr")
#outputReader = OutputReader(r"E:\03_Archive\250827_JG BELT NOISE\ansys_motion_modeling\REPS_Gear_RF_Automation_JG_PROTO_new_contact_r2.dfr")
#outputReader = OutputReader(r"E:\03_Archive\250827_JG BELT NOISE\ansys_motion_modeling\REPS_Gear_RF_Automation_JG_T_CAR_30_deg_new_contact_r2.dfr")
outputReader = OutputReader(r"E:\03_Archive\250827_JG BELT NOISE\ansys_motion_modeling\REPS_Gear_RF_Automation_JG_PROTO_r2_87mm.dfr")


# Get Curves
paths = List[str]()
#paths.Add("Acceleration/X")
paths.Add("Force on Base Marker/Magnitude")

plotParameter = PlotParameters()
plotParameter.PlotDataType = PlotDataType.PlotResult
plotParameter.AutoDecimation = False
#plotParameter.Target = "TIE_ROD_R"
plotParameter.Target = "FJ_Motor_Pulley"
plotParameter.Paths = paths

results = outputReader.GetCurves(plotParameter)

x=[]
y=[]

for kvp in results:
    path_name = kvp.Key
    points = kvp.Value

    for pt in points:
        x.append(pt.X)
        y.append(pt.Y)

# Close
outputReader.Close()

import numpy as np
from scipy.fft import fft, fftfreq

# Select data within the 0.3s to 1s range
start_time = 0.3
end_time = 1.0

# Find indices within the desired time range
indices = [i for i, time in enumerate(x) if start_time <= time <= end_time]

# Extract the corresponding x and y values
x_subset = [x[i] for i in indices]
y_subset = [y[i] for i in indices]

# Since the time interval is not uniform, interpolate the data to create a uniformly sampled signal for FFT
# First, create a uniformly spaced time vector
num_points = len(x_subset)  # Or choose a different number of points for interpolation
time_uniform = np.linspace(x_subset[0], x_subset[-1], num_points)

# Interpolate y_subset onto the uniform time vector
y_uniform = np.interp(time_uniform, x_subset, y_subset)

# Calculate the sampling frequency assuming the interpolated data is uniformly sampled
# dt is the average time difference in the original subset
dt = np.mean(np.diff(x_subset))
# Fs is the sampling frequency
Fs = 1/dt

# Perform FFT
yf = fft(y_uniform)
xf = fftfreq(num_points, dt)[:num_points//2]

import pandas as pd

# Create DataFrames for raw data and FFT results
raw_data_df = pd.DataFrame({'Time (s)': x, 'Value': y})
fft_results_df = pd.DataFrame({'Frequency (Hz)': xf, 'Amplitude': 2.0/num_points * np.abs(yf[0:num_points//2])})

# Combine the two DataFrames
# We'll use concat and add a key to distinguish between raw data and FFT results
# Since they have different number of rows, we'll pad with NaN values
combined_df = pd.concat([raw_data_df, fft_results_df], axis=1)


# Save the combined DataFrame to a single CSV file
combined_df.to_csv('combined_data_and_fft_results.csv', index=False)

print("Raw data and FFT results saved to combined_data_and_fft_results.csv")

Raw data and FFT results saved to combined_data_and_fft_results.csv
